In [ ]:
pip install tensorflow tensorflow-federated

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of tensorflow-federated to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

# Generate synthetic data for two hospitals (Hospital A and Hospital B)
def generate_synthetic_data(num_samples, seed=0):
    np.random.seed(seed)
    x = np.random.rand(num_samples, 10)  # 10 features
    y = np.random.randint(0, 2, num_samples)  # Binary classification target
    return pd.DataFrame(x, columns=[f'feature_{i}' for i in range(10)]), pd.Series(y, name='target')

# Generate data for two hospitals
hospital_A_data, hospital_A_labels = generate_synthetic_data(5, seed=1)
hospital_B_data, hospital_B_labels = generate_synthetic_data(5, seed=2)

# Combine data as if it were centralized (Normal Data)
centralized_data = pd.concat([hospital_A_data, hospital_B_data])
centralized_labels = pd.concat([hospital_A_labels, hospital_B_labels])

# Display centralized data
print("Centralized Data (Normal Data):")
print(pd.concat([centralized_data, centralized_labels], axis=1))

# Create TensorFlow Datasets for Federated Learning
hospital_A_dataset = tf.data.Dataset.from_tensor_slices((hospital_A_data.values, hospital_A_labels.values)).batch(2)
hospital_B_dataset = tf.data.Dataset.from_tensor_slices((hospital_B_data.values, hospital_B_labels.values)).batch(2)

# Federated data is represented as a list of local datasets
federated_data = [hospital_A_dataset, hospital_B_dataset]

# Display federated data samples
print("\nFederated Data Sample from Hospital A:")
for x, y in hospital_A_dataset.take(1):
    print(f"Features: {x.numpy()}, Labels: {y.numpy()}")

print("\nFederated Data Sample from Hospital B:")
for x, y in hospital_B_dataset.take(1):
    print(f"Features: {x.numpy()}, Labels: {y.numpy()}")


In [ ]:
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
import pandas as pd

In [ ]:
# Generate synthetic data for two hospitals (Hospital A and Hospital B)
def generate_synthetic_data(num_samples, seed=0):
    np.random.seed(seed)
    x = np.random.rand(num_samples, 10)  # 10 features
    y = np.random.randint(0, 2, num_samples)  # Binary classification target
    return pd.DataFrame(x, columns=[f'feature_{i}' for i in range(10)]), pd.Series(y, name='target')

In [ ]:
# Generate data for two hospitals
hospital_A_data, hospital_A_labels = generate_synthetic_data(5, seed=1)
hospital_B_data, hospital_B_labels = generate_synthetic_data(5, seed=2)

In [ ]:
print(hospital_A_data, hospital_A_labels)

   feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
0   0.417022   0.720324   0.000114   0.302333   0.146756   0.092339   
1   0.419195   0.685220   0.204452   0.878117   0.027388   0.670468   
2   0.800745   0.968262   0.313424   0.692323   0.876389   0.894607   
3   0.098347   0.421108   0.957890   0.533165   0.691877   0.315516   
4   0.988861   0.748166   0.280444   0.789279   0.103226   0.447894   

   feature_6  feature_7  feature_8  feature_9  
0   0.186260   0.345561   0.396767   0.538817  
1   0.417305   0.558690   0.140387   0.198101  
2   0.085044   0.039055   0.169830   0.878143  
3   0.686501   0.834626   0.018288   0.750144  
4   0.908596   0.293614   0.287775   0.130029   0    1
1    1
2    1
3    0
4    0
Name: target, dtype: int64


In [ ]:
# Combine data as if it were centralized (Normal Data)
centralized_data = pd.concat([hospital_A_data, hospital_B_data])
centralized_labels = pd.concat([hospital_A_labels, hospital_B_labels])

# Display centralized data (Normal Data)
print("Centralized Data (Normal Data):")
print(pd.concat([centralized_data, centralized_labels], axis=1))

Centralized Data (Normal Data):
   feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
0   0.417022   0.720324   0.000114   0.302333   0.146756   0.092339   
1   0.419195   0.685220   0.204452   0.878117   0.027388   0.670468   
2   0.800745   0.968262   0.313424   0.692323   0.876389   0.894607   
3   0.098347   0.421108   0.957890   0.533165   0.691877   0.315516   
4   0.988861   0.748166   0.280444   0.789279   0.103226   0.447894   
0   0.435995   0.025926   0.549662   0.435322   0.420368   0.330335   
1   0.621134   0.529142   0.134580   0.513578   0.184440   0.785335   
2   0.505246   0.065287   0.428122   0.096531   0.127160   0.596745   
3   0.467787   0.201743   0.640407   0.483070   0.505237   0.386893   
4   0.964551   0.500008   0.889520   0.341614   0.567144   0.427546   

   feature_6  feature_7  feature_8  feature_9  target  
0   0.186260   0.345561   0.396767   0.538817       1  
1   0.417305   0.558690   0.140387   0.198101       1  
2   0.085044   0.0

In [ ]:
# Create TensorFlow Datasets for Federated Learning
#This function converts the input data (features and labels) into a tf.data.Dataset object. This conversion allows TensorFlow to manage data loading and batching efficiently.
hospital_A_dataset = tf.data.Dataset.from_tensor_slices((hospital_A_data.values, hospital_A_labels.values)).batch(2)
hospital_B_dataset = tf.data.Dataset.from_tensor_slices((hospital_B_data.values, hospital_B_labels.values)).batch(2)

# Federated data is represented as a list of local datasets
federated_data = [hospital_A_dataset, hospital_B_dataset]

In [ ]:
# Display federated data samples to demonstrate privacy
# The list federated_data is used as input to the federated learning process. It represents the decentralized data stored on each client.
print("\nFederated Data (Data is kept locally):")
print("Hospital A Data Sample:")
for x, y in hospital_A_dataset.take(1):
    print(f"Features: {x.numpy()}, Labels: {y.numpy()}")

print("\nHospital B Data Sample:")
for x, y in hospital_B_dataset.take(1):
    print(f"Features: {x.numpy()}, Labels: {y.numpy()}")


Federated Data (Data is kept locally):
Hospital A Data Sample:
Features: [[4.17022005e-01 7.20324493e-01 1.14374817e-04 3.02332573e-01
  1.46755891e-01 9.23385948e-02 1.86260211e-01 3.45560727e-01
  3.96767474e-01 5.38816734e-01]
 [4.19194514e-01 6.85219500e-01 2.04452250e-01 8.78117436e-01
  2.73875932e-02 6.70467510e-01 4.17304802e-01 5.58689828e-01
  1.40386939e-01 1.98101489e-01]], Labels: [1 1]

Hospital B Data Sample:
Features: [[0.4359949  0.02592623 0.54966248 0.43532239 0.4203678  0.33033482
  0.20464863 0.61927097 0.29965467 0.26682728]
 [0.62113383 0.52914209 0.13457995 0.51357812 0.18443987 0.78533515
  0.85397529 0.49423684 0.84656149 0.07964548]], Labels: [0 1]


In [ ]:
# Define a model function using TFF's tff.learning.from_keras_model
# federated learning environment using TensorFlow Federated (TFF).
# model_fn: This function defines the machine learning model that will be used for federated learning.
# The model_fn function provides TFF with a blueprint for creating the model on each client.
# This function is called by TFF for every client participating in federated learning to ensure that the same model architecture is used across all clients.

def model_fn():
    keras_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(10,)),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    # This function converts the Keras model (keras_model) into a format that TensorFlow Federated (TFF) can use for federated learning.
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=hospital_A_dataset.element_spec, # input_spec: Specifies the structure of the input data (e.g., shape and data type).
                                                    # This is required for TFF to understand the format of the data coming from each client.
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

In [ ]:
# Create a federated training process using the new API
# tff.learning.algorithms.build_weighted_fed_avg: This function builds a federated training process using the Federated Averaging (FedAvg) algorithm.
# FedAvg is one of the most common algorithms for federated learning, where each client computes updates to the model parameters locally,
# and a central server averages these updates to improve the global model.
# This line sets up the entire federated learning process. It defines how the model will be trained across multiple clients and how the local
# model updates will be aggregated on the server.

trainer = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.01)   #Specifies the optimizer to use on each client for local training
)

# Initialize the federated learning process
# This function initializes the federated learning process and returns the initial state of the federated computation.
# The state includes the initial model parameters (weights and biases) and any other information needed to begin the training process.
state = trainer.initialize()

# Simulate multiple rounds of federated training
# Each round consists of local training on each client followed by aggregation of updates on the server.
# The number of rounds (NUM_ROUNDS) determines how many times this process will repeat.
NUM_ROUNDS = 100 #This line sets the number of training rounds (iterations) for federated learning.


for round_num in range(1, NUM_ROUNDS + 1):
    state, metrics = trainer.next(state, federated_data)  #federated_data: The list of local datasets for each client (e.g., Hospital A and Hospital B).
                                                          #  This data is used by each client to perform local training.
    print(f'Round {round_num}, Metrics: {metrics}')


Round 1, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.5), ('loss', 0.8199951), ('num_examples', 10), ('num_batches', 6)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Round 2, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.5), ('loss', 0.81347895), ('num_examples', 10), ('num_batches', 6)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Round 3, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.5), ('loss', 0.8083316), ('num_examples', 10), ('num_batches', 6)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Round 4, Metric